# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 19 2023 11:09AM,261411,10,Enova-11686,Emerginnova,Released
1,Apr 19 2023 11:08AM,261406,20,TR-ROM-040002023,Granules Consumer Health,Released
2,Apr 19 2023 11:07AM,261404,15,9386994,"Brookfield Pharmaceuticals, LLC",Released
3,Apr 19 2023 11:07AM,261404,15,9387613,"Brookfield Pharmaceuticals, LLC",Released
4,Apr 19 2023 11:07AM,261404,15,9387625,"Brookfield Pharmaceuticals, LLC",Released
5,Apr 19 2023 11:07AM,261404,15,9391494,"Brookfield Pharmaceuticals, LLC",Released
6,Apr 19 2023 11:07AM,261404,15,9391495,"Brookfield Pharmaceuticals, LLC",Released
7,Apr 19 2023 11:07AM,261402,15,VT80186282,"Virtus Pharmaceuticals, LLC",Released
8,Apr 19 2023 11:07AM,261402,15,VT80186283,"Virtus Pharmaceuticals, LLC",Released
9,Apr 19 2023 11:07AM,261402,15,VT80186306,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,261406,Released,1
29,261408,Released,1
30,261409,Released,1
31,261410,Released,9
32,261411,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261406,NaN,1.0
261408,NaN,1.0
261409,NaN,1.0
261410,NaN,9.0
261411,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
260869,6.0,27.0
260910,8.0,5.0
261117,9.0,1.0
261321,0.0,25.0
261331,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
260869,6,27
260910,8,5
261117,9,1
261321,0,25
261331,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,260869,6,27
1,260910,8,5
2,261117,9,1
3,261321,0,25
4,261331,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,260869,6,27
1,260910,8,5
2,261117,9,1
3,261321,,25
4,261331,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 19 2023 11:09AM,261411,10,Emerginnova
1,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health
2,Apr 19 2023 11:07AM,261404,15,"Brookfield Pharmaceuticals, LLC"
7,Apr 19 2023 11:07AM,261402,15,"Virtus Pharmaceuticals, LLC"
12,Apr 19 2023 11:06AM,261401,15,"Brookfield Pharmaceuticals, LLC"
14,Apr 19 2023 11:05AM,261410,20,Reliable 1 Laboratories LLC
23,Apr 19 2023 10:47AM,261409,19,"Granules Pharmaceuticals, Inc."
24,Apr 19 2023 10:43AM,261408,12,"SD Head USA, LLC"
25,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC"
27,Apr 19 2023 10:26AM,261403,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 19 2023 11:09AM,261411,10,Emerginnova,,1
1,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health,,1
2,Apr 19 2023 11:07AM,261404,15,"Brookfield Pharmaceuticals, LLC",,5
3,Apr 19 2023 11:07AM,261402,15,"Virtus Pharmaceuticals, LLC",,5
4,Apr 19 2023 11:06AM,261401,15,"Brookfield Pharmaceuticals, LLC",,2
5,Apr 19 2023 11:05AM,261410,20,Reliable 1 Laboratories LLC,,9
6,Apr 19 2023 10:47AM,261409,19,"Granules Pharmaceuticals, Inc.",,1
7,Apr 19 2023 10:43AM,261408,12,"SD Head USA, LLC",,1
8,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC",,2
9,Apr 19 2023 10:26AM,261403,10,Emerginnova,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 11:09AM,261411,10,Emerginnova,1,
1,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health,1,
2,Apr 19 2023 11:07AM,261404,15,"Brookfield Pharmaceuticals, LLC",5,
3,Apr 19 2023 11:07AM,261402,15,"Virtus Pharmaceuticals, LLC",5,
4,Apr 19 2023 11:06AM,261401,15,"Brookfield Pharmaceuticals, LLC",2,
5,Apr 19 2023 11:05AM,261410,20,Reliable 1 Laboratories LLC,9,
6,Apr 19 2023 10:47AM,261409,19,"Granules Pharmaceuticals, Inc.",1,
7,Apr 19 2023 10:43AM,261408,12,"SD Head USA, LLC",1,
8,Apr 19 2023 10:39AM,261405,19,"NAPP Technologies, LLC",2,
9,Apr 19 2023 10:26AM,261403,10,Emerginnova,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 11:09AM,261411,10,Emerginnova,1,
1,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health,1,
2,Apr 19 2023 11:07AM,261404,15,"Brookfield Pharmaceuticals, LLC",5,
3,Apr 19 2023 11:07AM,261402,15,"Virtus Pharmaceuticals, LLC",5,
4,Apr 19 2023 11:06AM,261401,15,"Brookfield Pharmaceuticals, LLC",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 11:09AM,261411,10,Emerginnova,1.0,NaN
1,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health,1.0,NaN
2,Apr 19 2023 11:07AM,261404,15,"Brookfield Pharmaceuticals, LLC",5.0,NaN
3,Apr 19 2023 11:07AM,261402,15,"Virtus Pharmaceuticals, LLC",5.0,NaN
4,Apr 19 2023 11:06AM,261401,15,"Brookfield Pharmaceuticals, LLC",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 11:09AM,261411,10,Emerginnova,1.0,0.0
1,Apr 19 2023 11:08AM,261406,20,Granules Consumer Health,1.0,0.0
2,Apr 19 2023 11:07AM,261404,15,"Brookfield Pharmaceuticals, LLC",5.0,0.0
3,Apr 19 2023 11:07AM,261402,15,"Virtus Pharmaceuticals, LLC",5.0,0.0
4,Apr 19 2023 11:06AM,261401,15,"Brookfield Pharmaceuticals, LLC",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2091056,19.0,6.0
12,261408,1.0,0.0
15,1568256,58.0,8.0
19,1829666,71.0,4.0
20,1305712,43.0,23.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2091056,19.0,6.0
1,12,261408,1.0,0.0
2,15,1568256,58.0,8.0
3,19,1829666,71.0,4.0
4,20,1305712,43.0,23.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,19.0,6.0
1,12,1.0,0.0
2,15,58.0,8.0
3,19,71.0,4.0
4,20,43.0,23.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,12,Released,1.0
2,15,Released,58.0
3,19,Released,71.0
4,20,Released,43.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20
Status,,,,,
Executing,6.0,0.0,8.0,4.0,23.0
Released,19.0,1.0,58.0,71.0,43.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20
0,Executing,6.0,0.0,8.0,4.0,23.0
1,Released,19.0,1.0,58.0,71.0,43.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20
0,Executing,6.0,0.0,8.0,4.0,23.0
1,Released,19.0,1.0,58.0,71.0,43.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()